## Warning: 

This notebook wipes your Experiment history clean. Use only if you want to start fresh. You will lose all experiments, trials, trial components and hosted endpoints if you run this notebook.

You have been warned.

In [15]:
# !pip install -q --upgrade pip
# !pip install botocore<=1.18.0
# !pip install -q sagemaker smdebug boto3 awscli sagemaker-experiments --upgrade

/bin/sh: 1: cannot open =1.18.0: No such file


In [9]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [10]:
import time
import boto3
sess = boto3.Session()
sm = sess.client('sagemaker')

In [11]:
def cleanup_experiment(experiment):
    for trial_summary in experiment.list_trials():
        try:
            trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
            for trial_component_summary in trial.list_trial_components():
                tc = TrialComponent.load(
                    sagemaker_boto_client=sm,
                    trial_component_name=trial_component_summary.trial_component_name)
                try:
                    trial.remove_trial_component(tc)
                    print(f'Deleting trial component: {tc.trial_component_name}')
                    tc.delete()
                except:
                    time.sleep(0.4)
                    continue
            print(f'Deleting trial: {trial.trial_name}')
            trial.delete()
        except:
            continue
    print(f'Deleting experiment: {experiment.experiment_name}')
    experiment.delete()

#### List of existing experiments

In [12]:
exp_names = [exp.experiment_name for exp in Experiment.list(sagemaker_boto_client=sm)]
print('Current experiments:')
exp_names

Current experiments:


['cifar10-training-experiment-1594536537',
 'cifar10-training-experiment-1594439099',
 'cifar10-training-experiment-1594376402',
 'cifar10-training-experiment-1594373728',
 'cifar10-training-experiment-1594372317',
 'cifar10-training-experiment-1594369849',
 'cifar10-training-experiment-1594367238',
 'cifar10-training-experiment-1594364801',
 'cifar10-training-experiment-1594319658',
 'cifar10-training-experiment-1594293483',
 'cifar10-training-experiment-1594292780',
 'cifar10-training-experiment-1594292190',
 'cifar10-training-experiment-1594291841',
 'cifar10-training-experiment-1594291356',
 'cifar10-training-experiment-1594288710',
 'cifar10-training-experiment-1594285627',
 'cifar10-training-experiment-1594281513',
 'cifar10-training-experiment-1594244637',
 'cifar10-training-experiment-1593999463']

In [13]:
cleanup_experiment(Experiment.load("gtc-sagemaker-training-experiment-1"))

ResourceNotFound: An error occurred (ResourceNotFound) when calling the DescribeExperiment operation: Experiment 'arn:aws:sagemaker:us-west-2:453691756499:experiment/gtc-sagemaker-training-experiment-1' does not exist.

In [ ]:
# for exp in exp_names:
#     cleanup_experiment(Experiment.load(exp))

Deleting trial: gtc-sagemaker-single-gpu-training-1
Deleting experiment: gtc-sagemaker-training-experiment-1
Deleting trial component: cifar10-training-rmsprop-custom-120-1594536631-aws-training-job
Deleting trial component: TrialComponent-2020-07-12-065031-bodm
Deleting trial: trial-rmsprop-custom-120-1594536631
Deleting trial component: cifar10-training-rmsprop-custom-60-1594536627-aws-training-job
Deleting trial component: cifar10-training-rmsprop-resnet-120-1594536617-aws-training-job
Deleting trial component: cifar10-training-sgd-custom-60-1594536598-aws-training-job
Deleting trial component: TrialComponent-2020-07-12-064954-omqr
Deleting trial: trial-sgd-custom-30-1594536594
Deleting trial component: cifar10-training-sgd-resnet-120-1594536589-aws-training-job
Deleting trial component: TrialComponent-2020-07-12-064910-cpdz
Deleting trial component: cifar10-training-sgd-resnet-60-1594536585-aws-training-job
Deleting trial component: cifar10-training-sgd-resnet-30-1594536580-aws-tra

#### Delete unassigned trial components

In [ ]:
for tc in TrialComponent.list(sagemaker_boto_client=sm):
    try:
        sm.delete_trial_component(TrialComponentName=tc.trial_component_name)
        print(f'Deleted: {tc.trial_component_name}')
        time.sleep(0.4)
    except:
        continue

#### Delete all endpoints

In [ ]:
for en in sm.list_endpoints()['Endpoints']:
    try:
        print(en['EndpointName'])
        sm.delete_endpoint(EndpointName=en['EndpointName'])
        time.sleep(0.4)
    except:
        continue